In [ ]:
import RingDataset
from Models import CNNModel, RNNGenerator, Distiller, MLP, RNNModel
import os
import numpy as np
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import re
import time

In [ ]:
testset =  RingDataset.RingDataset('core4ToSlice3_test.pkl', threshold=42)

testloader = DataLoader(testset, batch_size=256, num_workers=4)
classifier_test = CNNModel(42, dim=256).cuda()

gen=RNNGenerator(42, scale=0.25, dim=16, drop=0.0)
assert os.path.isfile('./models/best_{}_{}.pth'.format('cmp', 16))
gen.load_state_dict(torch.load('./models/best_{}_{}.pth'.format('cmp', 16)))



In [ ]:
def shifter(arr, window=32):
    dup = arr[:,None,:].expand(arr.size(0), arr.size(1)+1, arr.size(1))
    dup2 = dup.reshape(arr.size(0), arr.size(1), arr.size(1)+1)
    shifted = dup2[:,:window,:-window]
    return shifted

In [ ]:
class RNNGen2(nn.Module):
    def __init__(self, gen):
        super().__init__()
        self.encoder = gen.encoder

        self.resblock = gen.resblock

        self.decoder = gen.decoder


    def forward(self, x, distill=False):

        encoded = self.encoder(x.permute(0,2,1)) #N,C,S -> N,S,C
        
        res, _ = self.resblock(encoded)
        out = encoded + res #N,S,C
        out = self.decoder(out).view(out.size(0),-1)
        #out = out + self.scale*torch.randn_like(out)
        #out = out + noise
        
        if distill:
            return torch.relu(out), (encoded, res, out)
        else: 
            return torch.relu(out)
gen2 = RNNGen2(gen)

In [ ]:
model = gen2
model.eval()

model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
model_fp32_prepared = torch.quantization.prepare(model)

# calibrate the prepared model to determine quantization parameters for activations
# in a real world setting, the calibration would be done with a representative dataset
input_fp32, _ = next(iter(testloader))
shifted = shifter(input_fp32)
model_fp32_prepared(shifted)

# Convert the observed model to a quantized model. This does several things:
# quantizes the weights, computes and stores the scale and bias value to be
# used with each activation tensor, and replaces key operators with quantized
# implementations.
model_int8 = torch.quantization.convert(model_fp32_prepared)

# run the model, relevant calculations will happen in int8




# run the model, relevant calculations will happen in int8
res = model_int8(shifted)

In [ ]:
torch.backends.quantized.supported_engines